# Calcul de la consommation globale

On commence par charger les données, on les affiche, on les trie et on fait une première synthèse des consommation par moteur et par vol.

L'idées est de construire une table résumant les données de chaque vol et la consommation correspondate de cahque moteur.

In [1]:
from glob import glob
import pandas as pd
import statsmodels.formula.api as smf
import tabata as tbt
from tabata import Opset
import matplotlib.pyplot as plt
%matplotlib inline

## Récupération et affichage des données.

On affiche les données de manière à mieux comprendre comment définir les phases de vol.
Pour simplifier les accès j'utilise l'Opset de tabata.

In [2]:
datafiles = glob('../data/*.h5')

In [3]:
sorting = lambda name: int(name[8:])
ds = Opset(datafiles[0], sortkey=sorting)
ds.plotc()

In [7]:
from ipywidgets import interact, widgets

def mydoubleplot(varname, record):
    df = ds[record]
    tbt.doubleplot(df[varname], df["ALT [ft]"], title="Vol {}".format(record))

interact(mydoubleplot, varname=ds.df.columns, record=widgets.IntSlider(0,0,len(ds)-1,1)) ; #range(len(ds))) ;

interactive(children=(Dropdown(description='varname', options=('ALT [ft]', 'EGT_1 [deg C]', 'EGT_2 [deg C]', '…

In [8]:
LB2KG = 0.453592  # kg/lb 
KDENS = 0.73      # kg/l
L = []
for file in datafiles:
    ac = int(file[-5:-3])
    ds = Opset(file,sortkey=sorting) # Ne pas oublier de trier les enregistrements.
    for df in ds:
        if len(df.columns)>0 and "ALT [ft]" in df.columns:
            alt = df["ALT [ft]"]
            Alt_max = max(alt)
            if Alt_max > 20000:
                mc = df["M [Mach]"]
                # Il faudra peut-être séparer la montée et la descente de la croisière.
                df1 = df[mc>0.1]
                # Quand il faudra rajouter les phases au sol.
                #i0 = df1.index[0]
                #i1 = df1.index[-1]
                #taxi_o = df[:i0-1]
                #taxi_i = df[i1+1:]
                Leg = len(df1)/3600.0 # En h
                Q1 = df1["Q_1 [lb/h]"].sum()/3600.0 # en lb
                Q2 = df1["Q_2 [lb/h]"].sum()/3600.0
                M1 = Q1*LB2KG # En kg
                M2 = Q2*LB2KG
                Masse = M1+M2
                C1 = M1/KDENS # En l
                C2 = M2/KDENS
                Mach_max = df1["M [Mach]"].max()
                L += [[ac, 1, Leg, Alt_max, Mach_max, Masse, C1]]
                L += [[ac, 2, Leg, Alt_max, Mach_max, Masse, C2]]
dft = pd.DataFrame(L,columns = ['AC', 'ENG', 'Leg', 'Alt_max', 'Mach_max', 'Masse', 'Conso'])

In [9]:
dft

,AC,ENG,Leg,Alt_max,Mach_max,Masse,Conso
0,1,1,1.619722,39439.264501,0.640898,3725.812542,2547.848842
1,1,2,1.619722,39439.264501,0.640898,3725.812542,2556.003956
2,1,1,1.698056,39457.811063,0.649114,3873.297269,2652.136497
3,1,2,1.698056,39457.811063,0.649114,3873.297269,2653.750172
4,1,1,1.548889,40825.620025,0.624464,3395.975242,2326.615381
...,...,...,...,...,...,...,...
5989,3,2,1.132222,37621.701406,0.591598,2481.130405,1724.592070
5990,3,1,1.423889,36471.814551,0.632681,2990.838764,2019.218569
5991,3,2,1.423889,36471.814551,0.632681,2990.838764,2077.820834
5992,3,1,1.110278,37134.854149,0.599814,2425.202981,1639.174987
